In [1]:
import datetime
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
!pip install pmdarima


sm_boto3 = boto3.client('sagemaker')

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print('Using bucket ' + bucket)

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python -m pip install --upgrade pip' command.
Using bucket sagemaker-us-east-1-754307369999


In [2]:
import pandas as pd
from sagemaker.sklearn.estimator import SKLearn
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.session import Session
!pip install xlrd
!pip install s3fs


sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import boto3

client = boto3.client('rds')
response = client.describe_db_instances(
    DBInstanceIdentifier='sa2-db'
)
print(response['DBInstances'][0]['Endpoint']['Address'])
hostname=response['DBInstances'][0]['Endpoint']['Address']
connection = pg.connect(user = 'postgres',
password = 'sa2dbroot',
host = str(hostname),
port = '5432',
database = 'yash_sa_schema')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
data = pd.read_sql_query('SELECT * FROM yash_sa_schema.sa_file_ingst_log ORDER BY sa_file_ingst_log.upld_ts DESC LIMIT 1',con=connection)
print(data['published_file_path'].iloc[0])
#print(data['data_src_nm'].iloc[0])
print(data['published_file_name'].iloc[0])
print(data['model'].iloc[0])
Type=data['file_upld_typ'].iloc[0]
model=data['model'].iloc[0]
DatasetPath=data['published_file_path'].iloc[0]
filename=data['published_file_name'].iloc[0]
if model=='Arema':
    !aws s3 cp s3://sa2-script-bucket/scripts/Model_scripts/ArimaAttmodel.py ./DataFolder/Arema.py
elif model=='lr':
    !aws s3 cp s3://sa2-script-bucket/scripts/Model_scripts/lrAttmodel.py ./DataFolder/lr.py
elif model=='Customer_Segmentation':
    import pandas as pd
    from datetime import timedelta
    from io import StringIO

    def process_execution_function(data, id_col, date_col, order_id_col, value_col):
        data[date_col] = pd.to_datetime(data[date_col])
        last_run_date = data[date_col].max() + timedelta(days=1)
        # Grouping by CustomerID
        data_process = data.groupby([id_col]).agg({
                date_col: lambda x: (last_run_date - x.max()).days,
                order_id_col: 'count',
                value_col: 'sum'})
        # Rename the columns 
        data_process.rename(columns={date_col: 'Recency',
                                 order_id_col: 'Frequency',
                                 value_col: 'MonetaryValue'}, inplace=True)


        r_labels = range(4, 0, -1); f_labels = range(1, 5)
        # Assign these labels to 4 equal percentile groups 
        r_groups = pd.qcut(data_process['Recency'].rank(method='first'), q=4, labels=r_labels, duplicates='drop')
        # Assign these labels to 4 equal percentile groups 
        f_groups = pd.qcut(data_process['Frequency'].rank(method='first'), q=4, labels=f_labels, duplicates='drop')
        # Create new columns R and F 
        data_process = data_process.assign(R = r_groups.values, F = f_groups.values)
        data_process.head()
        # Create labels for MonetaryValue
        m_labels = range(1, 5)
        # Assign these labels to four equal percentile groups 
        m_groups = pd.qcut(data_process['MonetaryValue'].rank(method='first'), q=4, labels=m_labels, duplicates='drop')
        # Create new column M
        data_process = data_process.assign(M = m_groups.values)
        return data_process


    def join_rfm(x): 
        return str(x['R']) + str(x['F']) + str(x['M'])

    #Specify the Path
    data=pd.read_csv(DatasetPath)
    data['OrderReceivedDate'] = pd.to_datetime(data['OrderReceivedDate'])
    #Change corresponding columns as required for RFM Analysis
    id_col='shiptoname'
    date_col='OrderReceivedDate'
    order_id_col='ordernumber'
    value_col='Adj_OrderNetValue'

    data_process=process_execution_function(data,id_col,date_col, order_id_col, value_col)

    data_process['R']=data_process.loc[:,"R"].apply(lambda x : 5-x)
    data_process['F']=data_process.loc[:,"F"].apply(lambda x : 5-x)
    data_process['M']=data_process.loc[:,"M"].apply(lambda x : 5-x)
    data_process['RFM'] = data_process.apply(join_rfm, axis=1)

    data_process['RFM'] = data_process['RFM'].astype(str)
    data_process['R'] = data_process['R'].astype(str)
    data_process['F'] = data_process['F'].astype(str)
    data_process['M'] = data_process['M'].astype(str)

    data_process.loc[:,"CustomerName"]=data_process.index
    data_process=data_process.reset_index(drop=True)

    data_process.loc[:,"Segment"]='Others'
    indices=data_process[data_process.loc[:,"R"]=='1'].index
    data_process.loc[indices, "Segment"]="Recent Customers"

    indices=data_process[data_process.loc[:,"F"]=='1'].index
    data_process.loc[indices, "Segment"]="Loyal Customers"

    indices=data_process[data_process.loc[:,"M"]=='1'].index
    data_process.loc[indices,"Segment"]="Big Spenders"

    indices=data_process[data_process.loc[:,"RFM"]=='111'].index
    data_process.loc[indices,"Segment"]="Best Customers"

    indices=data_process[data_process.loc[:,"RFM"]=='311'].index
    data_process.loc[indices,"Segment"]="Almost Lost Customers"

    indices=data_process[data_process.loc[:,"RFM"]=='411'].index
    data_process.loc[indices,"Segment"]="Lost Customers"

    indices=data_process[data_process.loc[:,"RFM"]=='444'].index
    data_process.loc[indices,"Segment"]="Lost Cheap Customers"
    #data_process.to_csv("s3://sagemaker-us-east-1-754307369999/Output/RFM Data.csv", index=None)
    StringData = StringIO() 
    data_process.to_csv(StringData)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket_name='sa2-train-bucket',key='Output/RfmOutput.csv').put(Body=StringData.getvalue())




elif model=='Churn':
    !aws s3 cp s3://sa2-script-bucket/scripts/Model_scripts/Churn.py ./DataFolder/Churn.py
    !aws s3 cp s3://sa2-script-bucket/scripts/Model_scripts/requirements.txt ./DataFolder/
    from sagemaker.tensorflow import TensorFlow
    script_path='Churn.py'
    model_name='Churn'
    tf_estimator = TensorFlow(entry_point=script_path,
                              source_dir='./DataFolder',
                              base_job_name=model_name+f"-parameterName-hidden-channels-{int(time.time())}",
                              train_instance_type="ml.m5.large", 
                              role=role,
                              train_instance_count=1, 
                              framework_version='1.12', 
                              py_version='py3',
                              script_mode=True,
                              #hyperparameters={'epochs': 1}
                              hyperparameters={'filename': filename}
                               )
    tf_estimator.fit(inputs={'train': DatasetPath })
    tf_estimator.deploy(endpoint_name=model+'-endpoint-ML',instance_type='ml.m5.large',initial_instance_count=1)
    

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python -m pip install --upgrade pip' command.
sa2-db.cvl1on5n4pwi.us-east-1.rds.amazonaws.com
s3://ptg-banking-poc/sa_published_zone/DS1/file_type=training/part-00000-e6cdbaa2-38c2-43b5-8e9e-d6fc4cc38eea.c000.csv
part-00000-e6cdbaa2-38c2-43b5-8e9e-d6fc4cc38eea.c000.csv
lr
download: s3://sagemaker-us-east-1-754307369999/Models/lrAttmodel.py to DataFolder/lr.py


In [3]:
!aws s3 cp s3://sa2-script-bucket/scripts/Model_scripts/requirements.txt ./DataFolder/

download: s3://sagemaker-us-east-1-754307369999/ExtraFiles/requirements.txt to DataFolder/requirements.txt


In [4]:
#!aws s3 cp s3://sagemaker-us-east-1-754307369999/Input/1500RowPublishedFile.csv /opt/ml/input/data/train/1500RowPublishedFile.csv

In [5]:
sklearn_estimator = SKLearn(entry_point=model+'.py',
                                    train_instance_type='ml.m5.large',
                                    framework_version='0.20.0',
                                    source_dir='./DataFolder',
                                    role=role,
                                    hyperparameters={'filename': filename },
                                    enable_sagemaker_metrics=True)
sklearn_estimator.fit({'train':DatasetPath})

                # Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = sklearn_estimator.deploy(endpoint_name=model+'-endpoint--Banking-SA',instance_type='ml.m5.large',initial_instance_count=1)
    
    

2020-05-08 06:45:53 Starting - Starting the training job...
2020-05-08 06:45:55 Starting - Launching requested ML instances......
2020-05-08 06:47:00 Starting - Preparing the instances for training...
2020-05-08 06:47:50 Downloading - Downloading input data......
2020-05-08 06:48:41 Training - Downloading the training image..2020-05-08 06:48:56,107 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-08 06:48:56,109 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-08 06:48:56,118 sagemaker_sklearn_container.training INFO     Invoking user training script.

2020-05-08 06:48:56 Training - Training image download completed. Training in progress.2020-05-08 06:49:11,884 sagemaker-containers INFO     Module lr does not provide a setup.py. 
Generating setup.py
2020-05-08 06:49:11,884 sagemaker-containers INFO     Generating setup.cfg
2020-05-08 06:49:11,885 sagemaker-containers INFO     Generating MANIFEST.in
2020-05